### What is Dashboard:
#### A dashboard is a visual interface that displays important data and information in a clear and organized way
#### Often using charts, graphs, and tables
#### It helps users monitor, analyze, and make decisions based on real-time or summary data


### How to build a dashboard
#### 1. Define the Goals (what's the purpose of the dashboard? Who is the audience (executives, analysts, non-tech users)?
#### 2. Define the Metrics/KPIs should be displayed
#### 3. Identify and Gather Data Sources - Data Pipeline/ETL
#### 4. Design the Layout and Build the dashboard



### ETL:
#### Extract: retrieving data from different sources(different server, API, Hadoop, CSV etc)
#### Transform: cleaning, validating, converting, and standardizing the extracted data
#### Load

In [1]:
#Business dashboard - sales performance dashboard for executives (need to be at high level)
#In a sales dashboard, you might see: 
#Total revenue this month (monthly sales performance)
#Top-selling products
#Customers Behavior
#Breakdown analysis - Sales by region, etc
#etc

In [2]:
#pip install sqlalchemy

In [3]:
import pandas as pd

In [4]:
categories = pd.read_csv("categories.csv")
cities = pd.read_csv("cities.csv")
countries = pd.read_csv("countries.csv")
customers = pd.read_csv("customers.csv")
employees = pd.read_csv("employees.csv")
products = pd.read_csv("products.csv")
sales = pd.read_csv("sales.csv")

In [5]:
#importing sql library
from sqlalchemy import create_engine
 
# create a reference
# for sql library
engine = create_engine('sqlite://',
                       echo=False)
 
# attach the data frame to the sql
# with a name of the table
# as "Employee_Data"
categories.to_sql('categories',
               con=engine)
cities.to_sql('cities',
               con=engine)
countries.to_sql('countries',
               con=engine)
customers.to_sql('customers',
               con=engine)
employees.to_sql('employees',
               con=engine)
products.to_sql('products',
               con=engine)
sales.to_sql('sales',
               con=engine)

In [6]:
pd.read_sql_query("""
SELECT * 
FROM sales
limit 5
""",engine)

,index,Unnamed: 0,SalesID,SalesPersonID,CustomerID,ProductID,Quantity,Discount,TotalPrice,SalesDate,TransactionNumber
0,0,0,1,6,27039,381,7,0.0,0.0,2018-02-05 07:38:25.430,FQL4S94E4ME1EZFTG42G
1,1,1,2,16,25011,61,7,0.0,0.0,2018-02-02 16:03:31.150,12UGLX40DJ1A5DTFBHB8
2,2,2,3,13,94024,23,24,0.0,0.0,2018-05-03 19:31:56.880,5DT8RCPL87KI5EORO7B0
3,3,3,4,8,73966,176,19,0.2,0.0,2018-04-07 14:43:55.420,R3DR9MLD5NR76VO17ULE
4,4,4,5,10,32653,310,9,0.0,0.0,2018-02-12 15:37:03.940,4BGS0Z5OMAZ8NDAFHHP3


## Explore and Clean the data

In [7]:
#check the time range
pd.read_sql_query("""
SELECT MIN(SalesDate),MAX(SalesDate)
FROM sales
""",engine)

,MIN(SalesDate),MAX(SalesDate)
0,2018-01-01 00:00:04.070,2018-05-09 23:59:59.400


In [8]:
#validate "TotalPrice" column
pd.read_sql_query("""
SELECT *
FROM sales
WHERE TotalPrice > 0
""",engine)

,index,Unnamed: 0,SalesID,SalesPersonID,CustomerID,ProductID,Quantity,Discount,TotalPrice,SalesDate,TransactionNumber


In [9]:
#check if any duplicates
pd.read_sql_query("""
SELECT SalesID, COUNT(*)
FROM sales
GROUP BY 1
HAVING COUNT(*) > 1
""",engine)

,SalesID,COUNT(*)


In [10]:
#check if any duplicates
pd.read_sql_query("""
SELECT 
ProductID,
COUNT(*)
FROM products 
GROUP BY 1
HAVING COUNT(*) > 1
""",engine)

,ProductID,COUNT(*)


In [11]:
#check if there's any missing value
pd.read_sql_query("""
SELECT *
FROM sales
WHERE SalesDate IS NULL
""",engine)

,index,Unnamed: 0,SalesID,SalesPersonID,CustomerID,ProductID,Quantity,Discount,TotalPrice,SalesDate,TransactionNumber
0,50,50,51,21,12478,215,4,0.2,0.0,None,H8B08JBXS1TOWBJH3XMS
1,227,227,228,18,70321,74,18,0.0,0.0,None,SJ5VWMCZHIC2XBUEHEZJ
2,295,295,296,21,71322,219,19,0.0,0.0,None,8IXG8EPHL6LSLY3264G9
3,347,347,348,9,92737,352,24,0.0,0.0,None,HXGMYR6J7HWJUOTKB19Q
4,355,355,356,18,48857,278,13,0.0,0.0,None,0KLH8NGRQAGYRCVM4IF8
...,...,...,...,...,...,...,...,...,...,...,...
67521,6757793,6757793,6757794,7,33051,412,9,0.0,0.0,None,ZBRWISI5P40E51OALP7T
67522,6757833,6757833,6757834,15,22090,276,6,0.0,0.0,None,LZ00FIAZFGL5DLREDX4Z
67523,6757969,6757969,6757970,12,34077,63,9,0.0,0.0,None,WYPPNCGY57O7F0WXY0G0
67524,6758033,6758033,6758034,9,24940,22,7,0.0,0.0,None,B8RYWQV5ZMCUCYVOJJD6


### 1.Monthly Sales Performance

#### Objective: 
Analyze sales performance within the four-month period to identify trends and patterns.
#### Tasks:
Calculate total sales(revenue) for each month.

Compare sales performance across different product categories each month.

In [12]:
pd.read_sql_query("""
SELECT * 
FROM sales
limit 1
""",engine)

,index,Unnamed: 0,SalesID,SalesPersonID,CustomerID,ProductID,Quantity,Discount,TotalPrice,SalesDate,TransactionNumber
0,0,0,1,6,27039,381,7,0.0,0.0,2018-02-05 07:38:25.430,FQL4S94E4ME1EZFTG42G


In [34]:
pd.read_sql_query("""
SELECT DISTINCT Discount
FROM sales
""",engine)

,Discount
0,0.0
1,0.2
2,0.1


In [13]:
pd.read_sql_query("""
SELECT * 
FROM products
limit 1
""",engine)

,index,Unnamed: 0,ProductID,ProductName,Price,CategoryID,Class,ModifyDate,Resistant,IsAllergic,VitalityDays
0,0,0,1,Flour - Whole Wheat,74.2988,3,Medium,2018-02-16 08:21:49.190,Durable,Unknown,0.0


In [14]:
#Calculate total sales(revenue) for each month
pd.read_sql_query("""

WITH cte AS(
SELECT 
STRFTIME('%m', SalesDate) as month,
s.ProductID,
Quantity,
Discount,
Price,
Price * Quantity as TotalPrice,
(Price * Quantity - Discount) as revenue
FROM sales s
LEFT JOIN products p ON s.ProductID = p.ProductID
WHERE SalesDate < '2018-05-01'
AND SalesDate IS NOT NULL
)

SELECT month,
SUM(revenue)/1000000 as revenue_M
FROM cte
GROUP BY 1
ORDER BY 1
""",engine)

,month,revenue_M
0,01,1062.429692
1,02,957.836136
2,03,1064.046023
3,04,1028.104270


In [15]:
# #Calculate total sales(revenue) for each month
# pd.read_sql_query("""

# SELECT * 
# FROM products
# WHERE ProductID NOT IN (SELECT DISTINCT ProductID FROM sales)
# """,engine)

In [16]:
categories.head(5)

,Unnamed: 0,CategoryID,CategoryName
0,0,1,Confections
1,1,2,Shell fish
2,2,3,Cereals
3,3,4,Dairy
4,4,5,Beverages


In [17]:
#Compare sales performance across different product categories each month.
pd.read_sql_query("""

WITH cte AS(
SELECT 
STRFTIME('%m', SalesDate) as month,
s.ProductID,
CategoryName,
Quantity,
Discount,
Price,
Price * Quantity as TotalPrice,
(Price * Quantity - Discount) as revenue
FROM sales s
LEFT JOIN products p ON s.ProductID = p.ProductID
LEFT JOIN categories c ON p.CategoryID = c.CategoryID
WHERE SalesDate < '2018-05-01'
AND SalesDate is not null
)

SELECT
month,
CategoryName,
SUM(revenue)/1000000 as revenue_M
FROM cte
GROUP BY 1,2
ORDER BY 1 ASC,3 DESC
""",engine)

,month,CategoryName,revenue_M
0,01,Confections,136.924040
1,01,Meat,120.707840
2,01,Poultry,107.984508
3,01,Cereals,104.329764
4,01,Snails,91.697969
5,01,Produce,90.173389
6,01,Beverages,89.739851
7,01,Dairy,86.597580
8,01,Seafood,81.040042
9,01,Grain,79.776598


In [18]:
#Advanced: WoW sales Analyst, compare sale week over week (YOY)
pd.read_sql_query("""

WITH cte AS(
SELECT 
STRFTIME('%W', SalesDate) as week,
SUM(Price * Quantity - Discount)/1000000 as revenue_M
FROM sales s
LEFT JOIN products p ON s.ProductID = p.ProductID
WHERE SalesDate < '2018-05-01'
AND SalesDate IS NOT NULL
GROUP BY 1
ORDER BY 1
),

s AS(
SELECT
week,
revenue_M as revenue_M_this_week,
LAG(revenue_M) OVER (ORDER BY week) as revenue_M_prev_week,
--LAG(revenue_M,2) OVER (ORDER BY week) as revenue_M_prev2_week,
LEAD(revenue_M) OVER (ORDER BY week) as revenue_M_next_week
FROM cte
)


SELECT 
week,
revenue_M_this_week,
revenue_M_prev_week,
(revenue_M_this_week - revenue_M_prev_week) as delta_wow
FROM s

""",engine)

,week,revenue_M_this_week,revenue_M_prev_week,delta_wow
0,01,240.012715,NaN,NaN
1,02,239.249506,240.012715,-0.763209
2,03,240.011477,239.249506,0.761971
3,04,240.623928,240.011477,0.612451
4,05,239.662875,240.623928,-0.961053
5,06,238.617986,239.662875,-1.044889
6,07,239.549516,238.617986,0.931530
7,08,239.991605,239.549516,0.442089
8,09,239.511399,239.991605,-0.480206
9,10,239.565064,239.511399,0.053666


### 2. Top Products Identification

#### Objective: 
Determine which products are the best and worst performers within the dataset timeframe.
#### Tasks:
Analyze sales quantity to identify high-demand products.

Rank products based on total sales revenue.

Select the top selling categorys for each month

Examine the impact of product classifications on sales performance.

In [19]:
#Analyze sales quantity to identify high-demand products(top 10 products).
pd.read_sql_query("""

SELECT 
ProductName,
SUM(Quantity) as Quantity
FROM sales s
LEFT JOIN products p ON s.ProductID = p.ProductID
WHERE SalesDate < '2018-05-01'
AND SalesDate IS NOT NULL
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10




""",engine)

,ProductName,Quantity
0,Longos - Chicken Wings,184268
1,Yoghurt Tubes,183315
2,Clam Nectar,182905
3,"Thyme - Lemon, Fresh",182663
4,Apricots - Dried,182658
5,"Beef - Chuck, Boneless",182613
6,Onion Powder,182519
7,Beef - Short Loin,182364
8,Black Currants,182354
9,Beef - Montreal Smoked Brisket,182285


In [20]:
#Rank products based on total sales revenue.
pd.read_sql_query("""

WITH cte AS(
SELECT 
STRFTIME('%m', SalesDate) as month,
s.ProductID,
ProductName,
CategoryName,
Quantity,
Discount,
Price,
Price * Quantity as TotalPrice,
(Price * Quantity - Discount) as revenue
FROM sales s
LEFT JOIN products p ON s.ProductID = p.ProductID
LEFT JOIN categories c ON p.CategoryID = c.CategoryID
WHERE SalesDate < '2018-05-01'
AND SalesDate IS NOT NULL
),

agg AS(
SELECT
ProductName,
SUM(revenue)/1000000 as revenue_M
FROM cte
GROUP BY 1
)


SELECT
ProductName,
revenue_M,
RANK() OVER (ORDER BY revenue_M DESC) as rank_sales
FROM agg
ORDER BY 2 DESC


""",engine)

,ProductName,revenue_M,rank_sales
0,Bread - Calabrese Baguette,17.914905,1
1,Shrimp - 31/40,17.800801,2
2,Puree - Passion Fruit,17.776076,3
3,Tia Maria,17.721877,4
4,Zucchini - Yellow,17.601870,5
...,...,...,...
447,Bread Fig And Almond,0.211196,448
448,"Sole - Dover, Whole, Fresh",0.123208,449
449,Pastry - Raisin Muffin - Mini,0.076458,450
450,Apricots - Halves,0.067418,451


In [36]:
#select the top selling categorys for each month


pd.read_sql_query("""

WITH cte AS(
SELECT 
STRFTIME('%m', SalesDate) as month,
s.ProductID,
CategoryName,
Quantity,
Discount,
Price,
Price * Quantity as TotalPrice,
(Price * Quantity - Discount) as revenue
FROM sales s
LEFT JOIN products p ON s.ProductID = p.ProductID
LEFT JOIN categories c ON p.CategoryID = c.CategoryID
WHERE SalesDate < '2018-05-01'
AND SalesDate IS NOT NULL
),

s AS(
SELECT 
month,
CategoryName,
SUM(revenue)/1000000 as revenue_M
FROM cte
GROUP BY 1,2
),

r AS(
SELECT
month,
CategoryName,
revenue_M,
RANK() over (PARTITION BY month ORDER BY revenue_M DESC) as rank_sales
FROM s
)


SELECT 
month,
CategoryName,
revenue_M
FROM r
WHERE rank_sales = 1


""",engine)

,month,CategoryName,revenue_M
0,01,Confections,136.924040
1,02,Confections,122.708857
2,03,Confections,136.664861
3,04,Confections,131.977416


In [22]:
products.head(5)

,Unnamed: 0,ProductID,ProductName,Price,CategoryID,Class,ModifyDate,Resistant,IsAllergic,VitalityDays
0,0,1,Flour - Whole Wheat,74.2988,3,Medium,2018-02-16 08:21:49.190,Durable,Unknown,0.0
1,1,2,Cookie Chocolate Chip With,91.2329,3,Medium,2017-02-12 11:39:10.970,Unknown,Unknown,0.0
2,2,3,Onions - Cippolini,9.1379,9,Medium,2018-03-15 08:11:51.560,Weak,False,111.0
3,3,4,"Sauce - Gravy, Au Jus, Mix",54.3055,9,Medium,2017-07-16 00:46:28.880,Durable,Unknown,0.0
4,4,5,Artichokes - Jerusalem,65.4771,2,Low,2017-08-16 14:13:35.430,Durable,True,27.0


In [23]:
#Examine the impact of product classifications on sales performance.
pd.read_sql_query("""

SELECT 
Class,
STRFTIME('%m', SalesDate) as month,
SUM(Quantity) as Quantity,
SUM(Price * Quantity - Discount)/1000000 as revenue_M
FROM sales s
LEFT JOIN products p ON s.ProductID = p.ProductID
WHERE SalesDate < '2018-05-01'
AND SalesDate IS NOT NULL
GROUP BY 1,2
ORDER BY 1,2


""",engine)

,Class,month,Quantity,revenue_M
0,High,01,6622695,359.045372
1,High,02,5972812,323.381341
2,High,03,6625028,359.788647
3,High,04,6398247,346.843675
4,Low,01,7074860,356.885310
5,Low,02,6386703,321.739879
6,Low,03,7075101,357.436614
7,Low,04,6848471,345.615124
8,Medium,01,7202899,346.499010
9,Medium,02,6503328,312.714915


In [24]:
#Advanced: MoM sales Analyst, compare sale month over month within each category

pd.read_sql_query("""

WITH cte AS(
SELECT 
CategoryName,
STRFTIME('%m', SalesDate) as month,
SUM(Price * Quantity - Discount)/1000000 as revenue_M
FROM sales s
LEFT JOIN products p ON s.ProductID = p.ProductID
LEFT JOIN categories c ON p.CategoryID = c.CategoryID
WHERE SalesDate < '2018-05-01'
AND SalesDate IS NOT NULL
GROUP BY 1,2
ORDER BY 1,2
),

s AS(
SELECT
CategoryName,
month,
revenue_M as revenue_M_this_month,
LAG(revenue_M) OVER (PARTITION BY CategoryName ORDER BY month) as revenue_M_prev_month,
--LAG(revenue_M,2) OVER (PARTITION BY CategoryName ORDER BY month) as revenue_M_prev2_month,
LEAD(revenue_M) OVER (PARTITION BY CategoryName ORDER BY month) as revenue_M_next_month
FROM cte
)


SELECT 
CategoryName,
month,
revenue_M_this_month,
revenue_M_prev_month,
(revenue_M_this_month - revenue_M_prev_month) as delta_mom
FROM s

""",engine)

,CategoryName,month,revenue_M_this_month,revenue_M_prev_month,delta_mom
0,Beverages,01,89.739851,NaN,NaN
1,Beverages,02,81.272309,89.739851,-8.467542
2,Beverages,03,90.129818,81.272309,8.857510
3,Beverages,04,86.792713,90.129818,-3.337105
4,Cereals,01,104.329764,NaN,NaN
5,Cereals,02,94.519219,104.329764,-9.810545
6,Cereals,03,105.310405,94.519219,10.791186
7,Cereals,04,101.348453,105.310405,-3.961952
8,Confections,01,136.924040,NaN,NaN
9,Confections,02,122.708857,136.924040,-14.215183


### 3. Customer Purchase Behavior

#### Objective: 
Understand how customers interact with products during the four-month period.
#### Tasks:
Segment customers based on their purchase frequency and total spend.

Identify repeat customers versus one-time buyers.

Analyze average order value.

In [25]:
#Segment customers based on their purchase frequency and total spend.
pd.read_sql_query("""

WITH cte AS(
SELECT 
s.CustomerID,
FirstName,
LastName,
COUNT(DISTINCT SalesID) as frequency,
SUM(Price * Quantity - Discount)/1000000 as revenue_M
FROM sales s
LEFT JOIN products p ON s.ProductID = p.ProductID
LEFT JOIN customers c ON s.CustomerID = c.CustomerID
WHERE SalesDate < '2018-05-01'
AND SalesDate IS NOT NULL
GROUP BY 1,2,3
),

customer_bucket AS(
select 
CustomerID,
FirstName,
LastName,
frequency,
NTILE(3) OVER(ORDER BY frequency DESC) as frequency_bucket,
revenue_M,
NTILE(3) OVER(ORDER BY revenue_M DESC) as total_spend_bucket
FROM cte
)


SELECT
CustomerID,
FirstName,
LastName,
CASE WHEN frequency_bucket = 1 AND total_spend_bucket = 1 THEN 'High_frequency_High_Spend'
     WHEN frequency_bucket = 1 AND total_spend_bucket = 2 THEN 'High_frequency_Mid_Spend'
     WHEN frequency_bucket = 1 AND total_spend_bucket = 3 THEN 'High_frequency_Low_Spend'
     WHEN frequency_bucket = 2 AND total_spend_bucket = 1 THEN 'Mid_frequency_High_Spend'
     WHEN frequency_bucket = 2 AND total_spend_bucket = 2 THEN 'Mid_frequency_Mid_Spend'
     WHEN frequency_bucket = 2 AND total_spend_bucket = 3 THEN 'Mid_frequency_Low_Spend'
     WHEN frequency_bucket = 3 AND total_spend_bucket = 1 THEN 'Low_frequency_High_Spend'
     WHEN frequency_bucket = 3 AND total_spend_bucket = 2 THEN 'Low_frequency_Mid_Spend'
     WHEN frequency_bucket = 3 AND total_spend_bucket = 3 THEN 'Low_frequency_Low_Spend'
END AS customer_segment
FROM customer_bucket



""",engine)

,CustomerID,FirstName,LastName,customer_segment
0,65983,Dewayne,Walsh,High_frequency_High_Spend
1,30004,Randal,Carson,High_frequency_Mid_Spend
2,94800,Wayne,Chan,High_frequency_High_Spend
3,38069,Patrice,Bradshaw,High_frequency_Mid_Spend
4,6154,Micah,Park,High_frequency_Low_Spend
...,...,...,...,...
98754,40643,Christie,Bryan,Low_frequency_Low_Spend
98755,36950,Robert,Sims,Low_frequency_Low_Spend
98756,50610,Arlene,Maddox,Low_frequency_Low_Spend
98757,30416,Michael,Aguilar,Low_frequency_Low_Spend


In [26]:
#Identify repeat customers versus one-time buyers, and calculated the repeat customer percentage.
pd.read_sql_query("""

WITH cte AS(
SELECT 
s.CustomerID,
FirstName,
LastName,
COUNT(DISTINCT SalesID) AS num_order
FROM sales s
LEFT JOIN products p ON s.ProductID = p.ProductID
LEFT JOIN customers c ON s.CustomerID = c.CustomerID
WHERE SalesDate < '2018-05-01'
AND SalesDate IS NOT NULL
GROUP BY 1,2,3
),

type as(
SELECT 
CustomerID,
FirstName,
LastName,
CASE WHEN num_order > 1 THEN 'Repeat Customers' ELSE 'One-time Customers' END AS customer_type
FROM cte
)

SELECT
COUNT(DISTINCT CASE WHEN customer_type = 'Repeat Customers' THEN CustomerID END)/
COUNT(DISTINCT CustomerID) as repeat_customer_percentage
FROM type



""",engine)

,repeat_customer_percentage
0,1


In [27]:
#Analyze average order value.
pd.read_sql_query("""


SELECT 
s.CustomerID,
FirstName,
LastName,
SUM(Price * Quantity - Discount)/COUNT(DISTINCT SalesID) as aov
FROM sales s
LEFT JOIN products p ON s.ProductID = p.ProductID
LEFT JOIN customers c ON s.CustomerID = c.CustomerID
WHERE SalesDate < '2018-05-01'
AND SalesDate IS NOT NULL
GROUP BY 1,2,3
ORDER BY 4 DESC




""",engine)

,CustomerID,FirstName,LastName,aov
0,98637,Rick,Hinton,1600.979590
1,96367,Janet,Houston,1593.586272
2,97404,Miguel,Bishop,1558.922653
3,95586,Lorena,Henson,1551.201250
4,95272,Lester,Franklin,1548.689609
...,...,...,...,...
98754,1120,Latasha,Sweeney,39.138482
98755,2488,Clayton,Hawkins,38.700331
98756,2937,Clayton,Larson,38.658702
98757,3539,Marcy,Stark,38.437559


### 4. Salesperson Effectiveness

#### Objective: 
Evaluate the performance of sales personnel in driving sales.
#### Tasks:
Calculate total sales attributed to each salesperson.

Identify top-performing and underperforming sales staff.

In [28]:
#Calculate total sales attributed to each salesperson.
pd.read_sql_query("""

SELECT 
SalesPersonID,
SUM(Price * Quantity - Discount)/1000000 as revenue
FROM sales s
LEFT JOIN products p ON s.ProductID = p.ProductID
WHERE SalesDate < '2018-05-01'
AND SalesDate IS NOT NULL
GROUP BY 1
ORDER BY 2 DESC

""",engine)

,SalesPersonID,revenue
0,21,180.298269
1,20,179.786983
2,13,179.667878
3,5,179.583627
4,4,179.420093
5,8,179.400927
6,7,179.249796
7,22,179.238505
8,14,179.230366
9,12,179.184814


In [29]:
#Identify top-performing and underperforming sales staff.
pd.read_sql_query("""

SELECT 
SalesPersonID,
SUM(Price * Quantity - Discount)/1000000 as revenue
FROM sales s
LEFT JOIN products p ON s.ProductID = p.ProductID
WHERE SalesDate < '2018-05-01'
AND SalesDate IS NOT NULL
GROUP BY 1
ORDER BY 2 DESC
LIMIT 5

""",engine)

,SalesPersonID,revenue
0,21,180.298269
1,20,179.786983
2,13,179.667878
3,5,179.583627
4,4,179.420093


### 5. Geographical Sales Insights

#### Objective: 
Explore how sales are distributed across different cities and countries within the dataset.
#### Tasks:
Map sales data to specific cities and countries to identify high-performing regions.

Compare sales volumes between various geographical areas.

In [30]:
pd.read_sql_query("""

SELECT
CityName,
CountryName,
SUM(Price * Quantity - Discount)/1000000 as revenue
FROM sales s
LEFT JOIN products p ON s.ProductID = p.ProductID
LEFT JOIN employees e ON s.SalesPersonID = e.EmployeeID
LEFT JOIN cities t ON e.CityID = t.CityID
LEFT JOIN countries u ON t.CountryID = u.CountryID
WHERE SalesDate < '2018-05-01'
AND SalesDate IS NOT NULL
GROUP BY 1,2
ORDER BY 3 DESC

""",engine)

,CityName,CountryName,revenue
0,Lubbock,United States,537.023130
1,Baltimore,United States,359.365579
2,Columbus,United States,358.077026
3,New Orleans,United States,354.903559
4,Indianapolis,United States,179.786983
5,Anchorage,United States,179.667878
6,Anaheim,United States,179.583627
7,Little Rock,United States,179.400927
8,Las Vegas,United States,179.238505
9,Jackson,United States,179.230366


## Define Mtrics and Create ETL for Dashboard

In [31]:
#High-level Sale Performance Dashboard
#by week analysis - data need to be at week level
#bucket/Filters: Customer, cities, employee, category, class
#Metrics: number of orders, revenue, AOV, etc 

# what the table structure need to be

In [32]:
#Sales Dashboard, by week, wow, mom compare, filter by category, cities, etc
pd.read_sql_query("""
WITH cte AS(
SELECT 
s.CustomerID,
FirstName,
LastName,
COUNT(DISTINCT SalesID) as frequency,
SUM(Price * Quantity - Discount)/1000000 as revenue_M
FROM sales s
LEFT JOIN products p ON s.ProductID = p.ProductID
LEFT JOIN customers c ON s.CustomerID = c.CustomerID
WHERE SalesDate IS NOT NULL
GROUP BY 1,2,3
),

customer_bucket AS(
SELECT
CustomerID,
FirstName,
LastName,
frequency,
NTILE(3) OVER(ORDER BY frequency DESC) as frequency_bucket,
revenue_M,
NTILE(3) OVER(ORDER BY revenue_M DESC) as total_spend_bucket
FROM cte
),


segment AS(
SELECT
CustomerID,
FirstName,
LastName,
CASE WHEN frequency_bucket = 1 AND total_spend_bucket = 1 THEN 'High_frequency_High_Spend'
     WHEN frequency_bucket = 1 AND total_spend_bucket = 2 THEN 'High_frequency_Mid_Spend'
     WHEN frequency_bucket = 1 AND total_spend_bucket = 3 THEN 'High_frequency_Low_Spend'
     WHEN frequency_bucket = 2 AND total_spend_bucket = 1 THEN 'Mid_frequency_High_Spend'
     WHEN frequency_bucket = 2 AND total_spend_bucket = 2 THEN 'Mid_frequency_Mid_Spend'
     WHEN frequency_bucket = 2 AND total_spend_bucket = 3 THEN 'Mid_frequency_Low_Spend'
     WHEN frequency_bucket = 3 AND total_spend_bucket = 1 THEN 'Low_frequency_High_Spend'
     WHEN frequency_bucket = 3 AND total_spend_bucket = 2 THEN 'Low_frequency_Mid_Spend'
     WHEN frequency_bucket = 3 AND total_spend_bucket = 3 THEN 'Low_frequency_Low_Spend'
END AS customer_segment
FROM customer_bucket
)


SELECT 
STRFTIME('%Y', SalesDate) as year,
STRFTIME('%m', SalesDate) as month,
STRFTIME('%W', SalesDate) as week,
STRFTIME('%w', SalesDate) as weekday,
customer_segment,
CityName,
CountryName,
e.Gender as employee_gender,
CategoryName,
Class,
COUNT(SalesID) as num_orders,
SUM(Quantity) as Quantity,
SUM(Discount) as Discount,
SUM(Price * Quantity) as TotalPrice,
SUM(Price * Quantity - Discount) as revenue
FROM sales s
LEFT JOIN products p ON s.ProductID = p.ProductID
LEFT JOIN categories c ON p.CategoryID = c.CategoryID
LEFT JOIN employees e ON s.SalesPersonID = e.EmployeeID
LEFT JOIN cities t ON e.CityID = t.CityID
LEFT JOIN countries u ON t.CountryID = u.CountryID
LEFT JOIN segment ON s.CustomerID = segment.CustomerID
WHERE SalesDate IS NOT NULL
GROUP BY 1,2,3,4,5,6,7,8,9



""",engine)

,year,month,week,weekday,customer_segment,CityName,CountryName,employee_gender,CategoryName,Class,num_orders,Quantity,Discount,TotalPrice,revenue
0,2018,01,01,0,High_frequency_High_Spend,Albuquerque,United States,M,Beverages,Low,33,677,1.4,35522.4438,35521.0438
1,2018,01,01,0,High_frequency_High_Spend,Albuquerque,United States,M,Cereals,Low,36,704,1.1,40564.6019,40563.5019
2,2018,01,01,0,High_frequency_High_Spend,Albuquerque,United States,M,Confections,Low,52,1059,1.9,51997.6212,51995.7212
3,2018,01,01,0,High_frequency_High_Spend,Albuquerque,United States,M,Dairy,Low,29,581,1.4,29308.1672,29306.7672
4,2018,01,01,0,High_frequency_High_Spend,Albuquerque,United States,M,Grain,Low,34,684,0.9,38500.8790,38499.9790
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
268184,2018,05,19,3,Mid_frequency_Mid_Spend,Tucson,United States,M,Poultry,Medium,29,377,0.7,14125.0843,14124.3843
268185,2018,05,19,3,Mid_frequency_Mid_Spend,Tucson,United States,M,Produce,High,20,276,0.7,11833.6962,11832.9962
268186,2018,05,19,3,Mid_frequency_Mid_Spend,Tucson,United States,M,Seafood,High,21,296,0.7,15010.4854,15009.7854
268187,2018,05,19,3,Mid_frequency_Mid_Spend,Tucson,United States,M,Shell fish,High,18,254,0.4,9458.6917,9458.2917


In [ ]:
#save to file
to_file = pd.read_sql_query("""
WITH cte AS(
SELECT 
s.CustomerID,
FirstName,
LastName,
COUNT(distinct SalesID) as frequency,
SUM(Price * Quantity - Discount)/1000000 as revenue_M
FROM sales s
LEFT JOIN products p ON s.ProductID = p.ProductID
LEFT JOIN customers c ON s.CustomerID = c.CustomerID
WHERE SalesDate < '2018-03-30'
AND SalesDate IS NOT NULL
GROUP BY 1,2,3
),

customer_bucket AS(
SELECT
CustomerID,
FirstName,
LastName,
frequency,
NTILE(3) OVER(ORDER BY frequency DESC) as frequency_bucket,
revenue_M,
NTILE(3) OVER(ORDER BY revenue_M DESC) as total_spend_bucket
FROM cte
),


segment AS(
SELECT
CustomerID,
FirstName,
LastName,
CASE WHEN frequency_bucket = 1 AND total_spend_bucket = 1 THEN 'High_frequency_High_Spend'
     WHEN frequency_bucket = 1 AND total_spend_bucket = 2 THEN 'High_frequency_Mid_Spend'
     WHEN frequency_bucket = 1 AND total_spend_bucket = 3 THEN 'High_frequency_Low_Spend'
     WHEN frequency_bucket = 2 AND total_spend_bucket = 1 THEN 'Mid_frequency_High_Spend'
     WHEN frequency_bucket = 2 AND total_spend_bucket = 2 THEN 'Mid_frequency_Mid_Spend'
     WHEN frequency_bucket = 2 AND total_spend_bucket = 3 THEN 'Mid_frequency_Low_Spend'
     WHEN frequency_bucket = 3 AND total_spend_bucket = 1 THEN 'Low_frequency_High_Spend'
     WHEN frequency_bucket = 3 AND total_spend_bucket = 2 THEN 'Low_frequency_Mid_Spend'
     WHEN frequency_bucket = 3 AND total_spend_bucket = 3 THEN 'Low_frequency_Low_Spend'
END AS customer_segment
FROM customer_bucket
)


SELECT 
STRFTIME('%Y', SalesDate) as year,
STRFTIME('%m', SalesDate) as month,
STRFTIME('%W', SalesDate) as week,
--STRFTIME('%U', SalesDate) as week,
--STRFTIME('%w', SalesDate) as weekday,
CASE WHEN STRFTIME('%w', SalesDate) = '0' THEN 6 ELSE strftime('%w', SalesDate) - 1 END as weekday,
customer_segment,
CityName,
CountryName,
e.Gender as employee_gender,
CategoryName,
Class,
COUNT(SalesID) as num_orders,
SUM(Quantity) as Quantity,
SUM(Discount) as Discount,
SUM(Price * Quantity) as TotalPrice,
SUM(Price * Quantity - Discount) as revenue
FROM sales s
LEFT JOIN products p ON s.ProductID = p.ProductID
LEFT JOIN categories c ON p.CategoryID = c.CategoryID
LEFT JOIN employees e ON s.SalesPersonID = e.EmployeeID
LEFT JOIN cities t ON e.CityID = t.CityID
LEFT JOIN countries u ON t.CountryID = u.CountryID
LEFT JOIN segment ON s.CustomerID = segment.CustomerID
WHERE SalesDate < '2018-03-30'
AND SalesDate IS NOT NULL
GROUP BY 1,2,3,4,5,6,7,8,9,10


""",engine)

In [ ]:
to_file.to_csv('output.csv') 